## PDF Yüklemesinin Gerçekleştirlimesi

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("calikusu.pdf")
data = loader.load()

len(data)

c:\Users\ruveyda.cetin\AppData\Local\miniconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


298

In [2]:
len(data)

298

## Veriyi parçalara ayırma(Chunking işlemi)

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Daha büyük chunk size kullan (daha az chunk = daha az token)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # 1000'den 2000'e çıkar
    chunk_overlap=200  # Overlap ekle (context korunması için)
)
docs = text_splitter.split_documents(data)

print(f"Total chunks: {len(docs)}")

Total chunks: 537


In [14]:
print(f"Number of documents after chunking: {len(docs)}")

Number of documents after chunking: 537


In [16]:
docs[7]

Document(metadata={'producer': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creationdate': '2018-06-07T15:24:58+00:00', 'author': 'Reşat Nuri Güntekin', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'title': 'Çalıkuşu', 'source': 'calikusu.pdf', 'total_pages': 298, 'page': 7, 'page_label': '8'}, page_content='Sör Aleksi, izahatını bitirdikten sonra bizi çalışmaya bırakmıştı.\nÖn sıraları süsleyen ağırbaşlı sınıf birincileri hemen işe koyulmuşlardı. Yanlarında olmadığım\nhalde ne yazdıklarını omuzları üzerinden okumuş gibi biliyordum: “İlk hâtıranı, sevgili anneciğimin\nküçük karyolamın üstüne eğilen müşfik altın sarısı başı, bana muhabbetle gülümseyen gök mavisi\ngözleridir,” tarzında şairane bir yalancılık... Hakikatte annecikler altın sarısı ve gök mavisinden\nbaşka renklerde de olabilirdi. Fakat sörlerde okuyan kızların kaleminden bu renklere boyanmak, o\nbiçareler için bir mecburiyet, bizim için bir usuldü.\nBana gelin

## OpenAI AI Embeddings'i Kullanarak Embedding Oluşturma İşlemi

In [17]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings

In [18]:
load_dotenv()

True

In [22]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # large yerine small
)

## ChromaDB Üzerine Kayıt İşlemi

In [23]:
from langchain_chroma import Chroma

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

# 1. Daha büyük chunk'lar oluştur
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)
docs = text_splitter.split_documents(data)
print(f"📝 Created {len(docs)} chunks")

# 2. Embedding oluştur
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. Batch processing ile vector store oluştur
def create_vector_store_in_batches(docs, embeddings, batch_size=40):
    vector_store = None
    total_batches = (len(docs) - 1) // batch_size + 1
    
    for i in tqdm(range(0, len(docs), batch_size), desc="Creating vector store"):
        batch = docs[i:i + batch_size]
        
        if vector_store is None:
            vector_store = Chroma.from_documents(
                documents=batch,
                embedding=embeddings,
                persist_directory="./chroma_db_openai"
            )
        else:
            vector_store.add_documents(batch)
    
    return vector_store

vector_store = create_vector_store_in_batches(docs, embeddings, batch_size=40)
print("✅ Vector store created and persisted")

📝 Created 607 chunks


Creating vector store: 100%|██████████| 16/16 [00:22<00:00,  1.40s/it]

✅ Vector store created and persisted


In [26]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":10})

In [27]:
retrieved_docs = retriever.invoke("What is encoder?")

In [28]:
len(retrieved_docs)

10

In [29]:
print(retrieved_docs[5].page_content)

müzakere etmek: Öğrencilerin ders hazırlamaları için çalışmaları.
müzakkere: (Müzekkere) Bir iş hakkında üst makama sunulan yazı.
müzmin: Uzun süreli.
-N-
nadide: Az görülür, değerli.
nadir: Seyrek, az.
nafîa: Bayındırlık.
nafile: Yararsız, boşa giden.
nalça: 1) Ayakkabılar çabuk eskimesin diye altına çakılan demir. 2) Katır, eşek, sığır gibi
hayvanların tırnakları altına çakılan demir parçası.
namünasip: Uygun olmayan.
nan: Ekmek.
nasihat: Öğüt.
nazın Bakan.
nedamet: Pişmanlık.
nefer: Asker.
nekahat: Hastalık sonrası sağlıklı duruma geçme dönemi.
nekin Bilmezlik.
neşretmek: Yaymak.
netice itibarıyla: Sonuç olarak.
netice: Sonuç.
nev'i: (Nevi) Çeşit, cins, tür.
nihayetinde: Sonunda.


## OpenAI API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir. 
- Orta Değerler(0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir. 
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı , ancak bazen tutarsız yanıtlar verebilir

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    max_tokens=500
)

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [32]:
system_prompt = (
    "📖 Sen, Reşat Nuri Güntekin’in *Çalıkuşu* romanı üzerine uzman, cana yakın bir edebi asistansın. "
    "Görevin, kısa ama etkileyici analizler yapmak; karakterleri, temaları ve duyguları samimi bir dille yorumlamak 😊 "
    "Sadece bu roman hakkında konuş 🛑 Başka kitaplardan bahsetme. "
    "Yanıtların Türkçe olsun 🇹🇷 ve açıklamalarını emojilerle zenginleştir 📌"
    "\n\n"
    "{context}"
)

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

## Soru-Cevap Zinciri Oluşturma ( LLM + PROMPT )

In [34]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

## RAG Zinciri Olutşurma ( RAG + LLM )

In [35]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

## Kullanıcı sorgusunu çalıştırma

In [ ]:
response = rag_chain.invoke({"input":"Peki bu kitabı özetlesen nasıl özetlersin?"})

In [24]:
print(response["answer"])

The Transformer architecture is a model that relies entirely on attention mechanisms, dispensing with recurrence and convolutions. It consists of an encoder and a decoder, both composed of stacks of identical layers. Each encoder layer has a multi-head self-attention mechanism and a position-wise feed-forward network, while each decoder layer includes these plus an additional multi-head attention layer over the encoder's output.
